In [1]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModel
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, metrics, regularizers
import pandas as pd
import numpy as np
import tensorflow as tf
import sys

/home/yu/kaggle-tweet-sentiment/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-30 01:24:47.295402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738218287.306451  101502 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738218287.309823  101502 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 01:24:47.321036: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

In [2]:
train = pd.read_csv("data/train.csv").dropna()
train, test = train_test_split(train, test_size=0.15)
print(train.info())
print(test.info())
# TODO see if selected_text needs to be fixed to word boundaries

<class 'pandas.core.frame.DataFrame'>
Index: 23358 entries, 16039 to 5268
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         23358 non-null  object
 1   text           23358 non-null  object
 2   selected_text  23358 non-null  object
 3   sentiment      23358 non-null  object
dtypes: object(4)
memory usage: 912.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 4122 entries, 7068 to 18854
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         4122 non-null   object
 1   text           4122 non-null   object
 2   selected_text  4122 non-null   object
 3   sentiment      4122 non-null   object
dtypes: object(4)
memory usage: 161.0+ KB
None


In [3]:
max_text_len = 128
inference_batch_size = 2000

def tokenize(texts, padding=True):
    padding = "max_length" if padding else "longest"
    return tokenizer(texts, padding=padding, max_length=max_text_len, return_tensors="tf")

def detokenize(ids, skip_special=True):
    return tokenizer.batch_decode(ids, skip_special_tokens=skip_special)

# https://stackoverflow.com/a/7100681
def rolling_window(a, size):
    shape = a.shape[:-1] + (a.shape[-1] - size + 1, size)
    strides = a.strides + (a. strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def find_span(texts, spans):
    texts = np.array(texts)
    spans = np.array(spans)[:, 1:]
    sizes = spans.argmin(axis=1) - 1
    span_ranges = np.zeros((texts.shape[0], 2))
    for i, text in enumerate(texts):
        window_size = sizes[i]
        matched_window = rolling_window(texts[i], window_size) == spans[i][:window_size]
        window_from = matched_window.all(axis=1).argmax()
        span_ranges[i, 0] = window_from
        span_ranges[i, 1] = window_from + window_size - 1
    return span_ranges

def eprint(*args, **kwargs):
    print(*args, **kwargs, file=sys.stderr)

def print_mem_info():
    info = tf.config.experimental.get_memory_info("/gpu:0")
    eprint(f"GPU memory usage {(info["current"] / info["peak"]):.2f}.")

def context_embeddings(texts, sentiments, selected_texts=None):
    text_tokens = tokenize(texts)
    text_embeddings = encoder(**text_tokens).last_hidden_state
    sentiment_tokens = tokenize(sentiments)
    sentiment_embeddings = encoder(**sentiment_tokens).last_hidden_state[:, 1:2, :]  # only need embedding for one word
    embeddings = tf.concat((text_embeddings, sentiment_embeddings), 1)
    if selected_texts is None:
        return embeddings
    else:
        selected_tokens = tokenize(selected_texts)
        targets = find_span(text_tokens["input_ids"], selected_tokens["input_ids"])
        return embeddings, targets

def dataset_generator(texts, sentiments, selected_texts=None):
    def gen():
        total = len(texts)
        yielded = 0
        while yielded < total:
            batch_i = yielded % inference_batch_size
            # batch inferencing
            if batch_i == 0:
                end = min(total, yielded + inference_batch_size)
                if selected_texts is None:
                    embeddings = context_embeddings(texts[yielded:end], sentiments[yielded:end])
                else:
                    embeddings, targets = context_embeddings(texts[yielded:end], sentiments[yielded:end], selected_texts[yielded:end])
            # feed the generator
            if selected_texts is None:
                yield embeddings[batch_i]
            else:
                yield embeddings[batch_i], (targets[batch_i, 0], targets[batch_i, 1])
            yielded += 1
    return gen

Naive implementation, flattened embeddings into FFN with one hidden layer.

In [4]:
inputs = keras.Input(shape=(1 + max_text_len, 768))
flat = layers.Flatten()(inputs)
hidden = layers.Dense(256, activation="relu", kernel_regularizer="l1l2", bias_regularizer="l2")(flat)
softmax_start = layers.Dense(max_text_len, activation="softmax", name="start", kernel_regularizer="l1l2", bias_regularizer="l2")(hidden)
softmax_end = layers.Dense(max_text_len, activation="softmax", name="end", kernel_regularizer="l1l2", bias_regularizer="l2")(hidden)
ffn = keras.Model(inputs=inputs, outputs=(softmax_start, softmax_end))
ffn.compile(
    optimizer=optimizers.Adam(),
    loss={"start": losses.SparseCategoricalCrossentropy(), "end": losses.SparseCategoricalCrossentropy()},
    metrics={"start": metrics.SparseCategoricalAccuracy(), "end": metrics.SparseCategoricalAccuracy()}
)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
encoder = TFAutoModel.from_pretrained("distilbert-base-uncased")
dataset_train = tf.data.Dataset.from_generator(
    dataset_generator(train.text.to_list(), train.sentiment.to_list(), train.selected_text.to_list()),
    output_signature=(tf.TensorSpec(shape=(1 + max_text_len, 768)), (tf.TensorSpec(shape=()), tf.TensorSpec(shape=())))
).shuffle(inference_batch_size)
dataset_test = tf.data.Dataset.from_generator(
    dataset_generator(test.text.to_list(), test.sentiment.to_list(), test.selected_text.to_list()),
    output_signature=(tf.TensorSpec(shape=(1 + max_text_len, 768)), (tf.TensorSpec(shape=()), tf.TensorSpec(shape=())))
)
history = ffn.fit(dataset_train.batch(64), epochs=10, validation_data=dataset_test.batch(64))

I0000 00:00:1738218288.983263  101502 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21856 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.

Epoch 1/10


I0000 00:00:1738218298.552538  101651 service.cc:148] XLA service 0x7ff380003ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738218298.552553  101651 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2025-01-30 01:24:58.585521: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1738218298.664574  101651 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-30 01:24:59.581972: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 104 bytes spill stores, 104 bytes spill loads

2025-01-30 01:24:59.620961: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1', 124 bytes sp

      5/Unknown 10s 39ms/step - end_loss: 10.5816 - end_sparse_categorical_accuracy: 0.0456 - loss: 24.2729 - start_loss: 13.6913 - start_sparse_categorical_accuracy: 0.2851

I0000 00:00:1738218300.128642  101651 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

2025-01-30 01:33:35.929406: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 16646503622005204375
2025-01-30 01:33:35.929435: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8541295726000041299
2025-01-30 01:33:35.929445: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2868698929229645346


365/365 ━━━━━━━━━━━━━━━━━━━━ 59s 141ms/step - end_loss: 0.4270 - end_sparse_categorical_accuracy: 0.8630 - loss: 0.9821 - start_loss: 0.5547 - start_sparse_categorical_accuracy: 0.8180 - val_end_loss: 2.8005 - val_end_sparse_categorical_accuracy: 0.5017 - val_loss: 4.9659 - val_start_loss: 2.1900 - val_start_sparse_categorical_accuracy: 0.5687
Epoch 10/10


2025-01-30 01:33:44.689057: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 16646503622005204375
2025-01-30 01:33:44.689120: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8541295726000041299
2025-01-30 01:33:44.689129: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2868698929229645346


365/365 ━━━━━━━━━━━━━━━━━━━━ 58s 140ms/step - end_loss: 0.3691 - end_sparse_categorical_accuracy: 0.8794 - loss: 0.8334 - start_loss: 0.4640 - start_sparse_categorical_accuracy: 0.8463 - val_end_loss: 2.9620 - val_end_sparse_categorical_accuracy: 0.5087 - val_loss: 5.2555 - val_start_loss: 2.3315 - val_start_sparse_categorical_accuracy: 0.5497


2025-01-30 01:34:43.031298: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 16646503622005204375
2025-01-30 01:34:43.031330: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8541295726000041299
2025-01-30 01:34:43.031340: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2868698929229645346


In [10]:
# https://www.kaggle.com/competitions/tweet-sentiment-extraction
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

score = 0
total = test.shape[0]
(y_start, y_end) = ffn.predict(dataset_test.batch(64))
y_start = tf.math.argmax(y_start, axis=1)
y_end = tf.math.argmax(y_end, axis=1)
for i in range(total):
    span_start = y_start[i]
    span_end = y_end[i]
    y_str = test.text.iloc[i][span_start:span_end]
    t_str = test.selected_text.iloc[i]
    score += 1 / total * jaccard(y_str, t_str)
score

65/65 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step


0.08917177226924795